__Example 7.15.__ Consider the spring model

$\ddot{z} + C\dot{z} + Kz = 0$

$z(0) = 2$, $\dot{z}(0) = -C$

with displacement observations so that

$y = \begin{bmatrix} 1 & 0 \end{bmatrix} \begin{bmatrix} z \\ \dot{z} \end{bmatrix} = z$.

The solution is 

$z(t) = 2e^{-Ct/2}\cos{\left( \sqrt{K-C^2/4} t\right)}$

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import Image
from scipy import stats
import numpy as np

In [ ]:
K = 20.5
C0 = 1.5
sigma = 0.1

In [ ]:
def f(t,C,K):
    return 2*exp(-C*t/2)*cos(sqrt(K-C**2/4)*t)

In [ ]:
N = 501
t = linspace(0, 5, N)
fig, ax  = plt.subplots()
ax.plot(t, f(t,C0,K));
data = f(t,C0,K) + np.random.normal(0, sigma, N)
ax.plot(t, data,'ro',markersize=1)

In [ ]:
def SSE(C):
    return sum( (2*exp(-C*t/2)*cos(sqrt(K-C**2/4)*t)-data)**2 )

In [ ]:
from scipy import optimize
C_OLS=optimize.fmin(SSE,1)
C_OLS

In [ ]:
fig, ax  = plt.subplots()
ax.plot(t, f(t,C_OLS,K)-data,'.');

$$\dfrac{\partial y}{\partial C} = e^{-C t/2} \left[ \dfrac{Ct}{\sqrt{4K-C^2}} \sin \left( t \sqrt{K - C^2/4}\right) - t \cos \left( t \sqrt{K - C^2/4}\right) \right]$$

In [ ]:
C = C_OLS
w = sqrt(K - C**2/4)
dydC = exp(-C*t/2)*(C*t/sqrt(4*K-C**2)*sin(w*t)+t*cos(w*t))

$V = \sigma_C^2 = \sigma_0^2 [ \chi(q)^T \chi(q) ]^{-1}$, $\chi(q) = \left[ \dfrac{\partial y}{\partial C} (t_1,q),\dots,\dfrac{\partial y}{\partial C} (t_N,q) \right]^T$

In [ ]:
V = sigma**2/np.dot(dydC,dydC)
V

95% confidence interval is:

In [ ]:
C_OLS + 1.96*sqrt(V)*np.array([-1,1])
C_OLS + stats.t.ppf(0.975,df=N-1)*sqrt(V)*np.array([-1,1])

Note that the _interval_ is random. How often is the true value in the interval?
If we repeat the problem (not possible in practice), we can check how often the true value is within the interval.

In [ ]:
mc = 1000
Csamples = np.zeros(mc)
InInterval = np.zeros(mc)
for imc in range(0,mc):
    data = f(t,C0,K) + np.random.normal(0, sigma, N)
    Csamples[imc]=optimize.fmin(SSE,1.5,disp=0)
    C = Csamples[imc]
    w = sqrt(K - C**2/4)
    dydC = exp(-C*t/2)*(C*t/sqrt(4*K-C**2)*sin(w*t)+t*cos(w*t))
    V = sigma**2/np.dot(dydC,dydC)
    InInterval[imc] = (C0 < C+1.96*sqrt(V)) & (C0 > C-1.96*sqrt(V))

In [ ]:
np.mean(InInterval)

In [ ]:
fig, ax  = plt.subplots()
ax.plot(Csamples);

In [ ]:
fig, ax  = plt.subplots()
plt.hist(Csamples, color = 'blue', edgecolor = 'black',bins = int(180/5))

In [ ]:
import seaborn as sns
sns.distplot(Csamples, hist=True, kde=True, 
             bins=int(180/5), color = 'blue',
             hist_kws={'edgecolor':'black'})

In [ ]:
np.mean(Csamples)

In [ ]:
np.std(Csamples)